In [30]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [31]:
def find_sheet_name(sheet_names):
    gdp_sheet_names = list(filter(lambda x: "gdp-ss" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(filter(lambda x: "gdp" in x.lower() and "ss" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(filter(lambda x: "gdp" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    if len(sheet_names) == 43:
        return "2"

    raise RuntimeError(sheet_names)

In [32]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "TỔNG SẢN PHẨM TRONG NƯỚC": "TỔNG SỐ",
    "Agriculture, Forestry and Fishing": "Nông, lâm nghiệp và thủy sản",
    "Agriculture": "Nông nghiệp",
    "Forestry": "Lâm nghiệp",
    "Fishery": "Thủy sản",
    "Industry and Construction": "Công nghiệp và xây dựng",
    "Industry": "Công nghiệp",
    "Mining and quarrying": "Khai khoáng",
    "Manufacturing": "Công nghiệp chế biến, chế tạo",
    "Production & supply of electricity, gas, hot water, steam and air-conditioning": "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí",
    "Water supply; sewerage, waste management and remediation activities": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Construction": "Xây dựng",
    "Services": "Dịch vụ",
    "Wholesales, retails, vehicles and motorcycles repairs": "Bán buôn và bán lẻ; sửa chữa ô tô, mô tô, xe máy và xe có động cơ khác",
    "Transportation and storage": "Vận tải, kho bãi",
    "Accommodation & catering services": "Dịch vụ lưu trú và ăn uống",
    "Information & communication": "Thông tin và truyền thông",
    "Finance, banking & insurance": "Hoạt động tài chính, ngân hàng và bảo hiểm",
    "Real estate business ": "Hoạt động kinh doanh bất động sản",
    "Scientific and technical specialty": "Hoạt động chuyên môn, khoa học và công nghệ",
    "Administrative activity and supporting service": "Hoạt động hành chính và dịch vụ hỗ trợ",
    "Activities of Communist Party, socio-political organizations; State management and national defense; compulsory social security": "Hoạt động của Đảng Cộng sản, tổ chức chính trị- xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc",
    "Education and training": "Giáo dục và đào tạo",
    "Health care & social relief": "Y tế và hoạt động trợ giúp xã hội",
    "Art, entertainment and recreation": "Nghệ thuật, vui chơi và giải trí",
    "Other service activities": "Hoạt động dịch vụ khác",
    "Activities of households as employers; undifferentiated goods and services producing activities of households for own use": "Hoạt động làm thuê các công việc trong các hộ gia đình, sản xuất sản phẩm vật chất và dịch vụ tự tiêu dùng của hộ gia đình",
    "Products taxes less subsidies on production": "Thuế sản phẩm trừ trợ cấp sản phẩm",
}

In [34]:
def get_gdp_ss(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df[df.columns[0]] = df[df.columns[0]].fillna(df[df.columns[1]])

    if len(df.columns) == 8:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[4],
                # df.columns[5],
                df.columns[6],
                df.columns[7],
            ]
        )
    elif len(df.columns) == 7:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[3],
                df.columns[4],
                # df.columns[6],
            ]
        )
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[4]])
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(
            columns=[df.columns[1], df.columns[2], df.columns[4], df.columns[5]]
        )
    else:
        raise RuntimeError(df.columns)

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ")
                    | (df[df.columns[0]] == "TOTAL")
                    | (df[df.columns[0]] == "TỔNG SẢN PHẨM TRONG NƯỚC")
                ].index[0]
            )
        )
    )

    df.columns = ["Ngành", "Tổng số", "YOY"]

    df[df["Ngành"] == "TỔNG SẢN PHẨM TRONG NƯỚC"] = "TỔNG SỐ"

    if len(df["Ngành"][df["Ngành"] == "TOTAL"].index) > 0:
        df["Ngành"] = df["Ngành"].map(en_to_vn)

    df = df[pd.to_numeric(df[df.columns[1]], errors="coerce").notnull()]

    df["YOY"] = df["YOY"] - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)
    return df


xl = pd.ExcelFile("datas/2023-03-29-02-Bieu-T3.2023-3.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_gdp_ss(xl, sheet_name, "2023-03-29")

,Ngành,Tổng số,YOY,Date
6,TỔNG SỐ,1320754.547853,3.318918,2023-03-29
7,"Nông, lâm nghiệp và thủy sản",140836.347594,2.524398,2023-03-29
8,Nông nghiệp,109002.057998,2.429005,2023-03-29
9,Lâm nghiệp,5757.283307,3.656374,2023-03-29
10,Thủy sản,26077.006289,2.676548,2023-03-29
11,Công nghiệp và xây dựng,459242.836583,-0.404306,2023-03-29
12,Công nghiệp,388692.936749,-0.820734,2023-03-29
13,Khai khoáng,39265.773624,-5.600361,2023-03-29
14,"Công nghiệp chế biến, chế tạo",297173.059846,-0.370883,2023-03-29
15,"Sản xuất và phân phối điện, khí đốt, nước nóng...",45134.405362,-0.318631,2023-03-29


In [35]:
import os
import contextlib

dfs_gdp_ss_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2018:
            continue

        if month not in [3, 6, 9, 12]:
            continue

        xl = pd.ExcelFile(file_path)

        df_gdp = get_gdp_ss(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_gdp_ss_origin.append(df_gdp)
    except Exception as e:
        print(file_name)

In [36]:
dfs_gdp_ss = pd.concat(dfs_gdp_ss_origin, ignore_index=True, axis=0)
dfs_gdp_ss["Date"] = pd.to_datetime(dfs_gdp_ss["Date"], dayfirst=False)
dfs_gdp_ss

,Ngành,Tổng số,YOY,Date
0,TỔNG SỐ,1643902.24118,5.469892,2023-12-29
1,"Nông, lâm nghiệp và thủy sản",184544.946662,4.304804,2023-12-29
2,Nông nghiệp,140131.988632,4.486614,2023-12-29
3,Lâm nghiệp,8660.257316,3.529904,2023-12-29
4,Thủy sản,35752.700714,3.918206,2023-12-29
...,...,...,...,...
666,Y tế và hoạt động trợ giúp xã hội,8381,6.97,2018-03-29
667,"Nghệ thuật, vui chơi và giải trí",4822,7.08,2018-03-29
668,Hoạt động dịch vụ khác,11510,5.78,2018-03-29
669,Hoạt động làm thuê các công việc trong các hộ ...,1113,6.1,2018-03-29


In [37]:
nganh = "TỔNG SỐ"
dfs_gdp_ss_per_nganh = dfs_gdp_ss[dfs_gdp_ss["Ngành"] == nganh]

dfs_gdp_ss_per_nganh["PCT_YOY"] = dfs_gdp_ss_per_nganh["Tổng số"].pct_change(-4)
print(dfs_gdp_ss_per_nganh)

       Ngành         Tổng số        YOY       Date   PCT_YOY
0    TỔNG SỐ   1643902.24118   5.469892 2023-12-29  0.068196
28   TỔNG SỐ  1450322.454378     4.0511 2023-09-29  0.053768
56   TỔNG SỐ  1407472.978003   3.275457 2023-06-29  0.045033
84   TỔNG SỐ  1320754.547853   3.318918 2023-03-29  0.052132
112  TỔNG SỐ  1538951.343532  13.714433 2022-12-29  0.048697
140  TỔNG SỐ   1376320.95164   7.828583 2022-09-29  0.145916
168  TỔNG SỐ  1346821.301552   5.052648 2022-06-29  0.078481
196  TỔNG SỐ  1255312.381476    5.03183 2022-03-29  0.055580
224  TỔNG SỐ  1467489.725843  -6.024775 2021-12-29  0.156975
252  TỔNG SỐ  1201065.954588   6.570692 2021-09-29  0.220931
280  TỔNG SỐ         1248813   4.650437 2021-06-29  0.406539
308  TỔNG SỐ    1189215.6721       4.48 2021-03-29  0.681529
336  TỔNG SỐ  1268385.274728   2.690682 2020-12-27  0.044754
364  TỔNG SỐ   983729.772117   0.387376 2020-09-29  0.027813
392  TỔNG SỐ   887862.394367    3.68211 2020-06-29  0.003792
420  TỔNG SỐ   707222.74

/tmp/ipykernel_9965/1353601895.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [38]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_gdp_ss_per_nganh[(dfs_gdp_ss_per_nganh["Ngành"] == "TỔNG SỐ")]

fig = px.area(df_show, x="Date", y="PCT_YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

